In [1]:
import time
from polygon import RESTClient
from datetime import datetime
import sqlite3
from pymongo import MongoClient

# Connect to SQLite database
conn_sqlite = sqlite3.connect(r'C:\Users\sai krishna\Desktop\Py Projects\Data Engineering\currency_conversion2603.db')
cursor_sqlite = conn_sqlite.cursor()

# Create SQLite table if not exists
cursor_sqlite.execute('''CREATE TABLE IF NOT EXISTS conversion_data (
                            id INTEGER PRIMARY KEY,
                            source_currency TEXT,
                            target_currency TEXT,
                            converted_value REAL,
                            timestamp TEXT,
                            writing_time TEXT
                        )''')

# Initialize MongoDB client and connect to the local database
client_mongodb = MongoClient('mongodb://localhost:27017/')
db_mongodb = client_mongodb['currency_conversion2603']
collection_mongodb = db_mongodb['conversion_data']

client_polygon = RESTClient('beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq')  # POLYGON_API_KEY

# Run for 2 hours (7200 seconds)
duration = 7200

for _ in range(duration):
    try:
        # Fetch conversion rates for EUR to USD
        EURUSD = client_polygon.get_real_time_currency_conversion("EUR", "USD")
        FXrate_EURUSD = EURUSD.converted
        FX_timestamp_EURUSD = datetime.utcfromtimestamp(EURUSD.last.timestamp / 1000)
        TS24_EURUSD = FX_timestamp_EURUSD.strftime('%Y-%m-%d %H:%M:%S')

        # Fetch conversion rates for JPY to USD
        JPYUSD = client_polygon.get_real_time_currency_conversion("JPY", "USD")
        FXrate_JPYUSD = JPYUSD.converted
        FX_timestamp_JPYUSD = datetime.utcfromtimestamp(JPYUSD.last.timestamp / 1000)
        TS24_JPYUSD = FX_timestamp_JPYUSD.strftime('%Y-%m-%d %H:%M:%S')

        # Fetch conversion rates for INR to USD
        INRUSD = client_polygon.get_real_time_currency_conversion("INR", "USD")
        FXrate_INRUSD = INRUSD.converted
        FX_timestamp_INRUSD = datetime.utcfromtimestamp(INRUSD.last.timestamp / 1000)
        TS24_INRUSD = FX_timestamp_INRUSD.strftime('%Y-%m-%d %H:%M:%S')

        # Get the current time (writing time)
        writing_time = datetime.utcnow()
        DB_time = writing_time.strftime('%Y-%m-%d %H:%M:%S')

        # Insert data into SQLite database
        cursor_sqlite.execute('''INSERT INTO conversion_data 
                                  (source_currency, target_currency, converted_value, timestamp, writing_time) 
                                  VALUES (?, ?, ?, ?, ?)''', 
                                  ('EUR', 'USD', FXrate_EURUSD, TS24_EURUSD, DB_time))
        cursor_sqlite.execute('''INSERT INTO conversion_data 
                                  (source_currency, target_currency, converted_value, timestamp, writing_time) 
                                  VALUES (?, ?, ?, ?, ?)''', 
                                  ('JPY', 'USD', FXrate_JPYUSD, TS24_JPYUSD, DB_time))
        cursor_sqlite.execute('''INSERT INTO conversion_data 
                                  (source_currency, target_currency, converted_value, timestamp, writing_time) 
                                  VALUES (?, ?, ?, ?, ?)''', 
                                  ('INR', 'USD', FXrate_INRUSD, TS24_INRUSD, DB_time))

        conn_sqlite.commit()

        # Insert data into MongoDB
        data_EURUSD = {
            'from': 'EUR',
            'to': 'USD',
            'converted_value': FXrate_EURUSD,
            'timestamp': TS24_EURUSD,
            'writing_time': DB_time
        }
        data_JPYUSD = {
            'from': 'JPY',
            'to': 'USD',
            'converted_value': FXrate_JPYUSD,
            'timestamp': TS24_JPYUSD,
            'writing_time': DB_time
        }
        data_INRUSD = {
            'from': 'INR',
            'to': 'USD',
            'converted_value': FXrate_INRUSD,
            'timestamp': TS24_INRUSD,
            'writing_time': DB_time
        }

        collection_mongodb.insert_one(data_EURUSD)
        collection_mongodb.insert_one(data_JPYUSD)
        collection_mongodb.insert_one(data_INRUSD)

        print(f"EUR to USD: Converted Value: {FXrate_EURUSD}, Timestamp: {TS24_EURUSD}")
        print(f"JPY to USD: Converted Value: {FXrate_JPYUSD}, Timestamp: {TS24_JPYUSD}")
        print(f"INR to USD: Converted Value: {FXrate_INRUSD}, Timestamp: {TS24_INRUSD}")

    except Exception as e:  # Handling potential errors
        print(f"Error fetching data: {e}")

    time.sleep(1)  # Delay for 1 sec

# Close the connections
conn_sqlite.close()
client_mongodb.close()


EUR to USD: Converted Value: 1.08, Timestamp: 2024-03-26 15:44:43
JPY to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:36
INR to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:44
EUR to USD: Converted Value: 1.08, Timestamp: 2024-03-26 15:44:46
JPY to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:36
INR to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:44
EUR to USD: Converted Value: 1.08, Timestamp: 2024-03-26 15:44:47
JPY to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:36
INR to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:44
EUR to USD: Converted Value: 1.08, Timestamp: 2024-03-26 15:44:48
JPY to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:36
INR to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:44
EUR to USD: Converted Value: 1.08, Timestamp: 2024-03-26 15:44:50
JPY to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:36
INR to USD: Converted Value: 0.01, Timestamp: 2024-03-26 15:44:44
EUR to USD

In [2]:


# Initialize MongoDB client
client = MongoClient('mongodb://localhost:27017/')
data_base = client['currency_conversion2603']
FX_rate = data_base['conversion_data']

# Define a dictionary to store sum and count
pairs = {'EURUSD': {'sum': 0, 'count': 0},
             'JPYUSD': {'sum': 0, 'count': 0},
             'INRUSD': {'sum': 0, 'count': 0}}

# Retrieve data from MongoDB and calculate avg
for data in FX_rate.find():
    currency_pair = data['from'] + data['to']
    converted_value = data['converted_value']
    pairs[currency_pair]['sum'] += converted_value
    pairs[currency_pair]['count'] += 1

average_data = {}
for pair, data in pairs.items():
    if data['count'] > 0:
        average_data[pair] = data['sum'] / data['count']
    else:
        average_data[pair] = 0

# average for each currency pair
for pair, average in average_data.items():
    print(f"Average for {pair}: {average}")
client.close()

Average for EURUSD: 1.0799999999999392
Average for JPYUSD: 0.00999999999999999
Average for INRUSD: 0.00999999999999999
